In [1]:
from functools import reduce
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

#### Get input as {key: value} pairs

In [2]:
with open("input.txt") as f:
    inp = f.readlines()
inp = {int(sp[0]):sp[1].strip() for sp in [line.split(": ") for line in inp]}
inp[507]

"P#qw:#/Vq:q-(.'qk iquq? V:Sq#Tq#!:nq wq!/ ,q/. eqd qss:q/ dqè. so)q. eq!?:-rà/:#eqwu#?q wq\\/:,/ dqV?o!  Tqàààw?#"

#### Generate cookies

In [3]:
try:
    with open("cookies.txt") as f:
        cookies = f.readlines()
        cookies_dict = {cookies[0].strip(): cookies[1]}
        
    cookies_dict
except FileNotFoundError:
    print("Generate your own cookies! (cookies.txt)")

In [4]:
def scrape_key(sender_identifier):
    """Uses html files if they exist, otherwise scrapes website
    to collect the decryption keys and user info"""
    
    if os.path.exists(f"html_files/{sender_identifier}.html"):
        with open(f"html_files/{sender_identifier}.html") as f:
            soup = BeautifulSoup(f, "html.parser")
            
    else:
        url = "https://aoc.lewagon.community/stats/users/" + str(sender_identifier)
        
        html = requests.get(url, cookies=cookies_dict).content
        soup = BeautifulSoup(html, "html.parser")
        html = soup.prettify("utf-8")
        with open(f"html_files/{sender_identifier}.html", "wb") as file:
            file.write(html)

    # Find name
    
    h2 = soup.find_all("h2", class_="text-lg strong text-center")
    info = f'{h2[0].text.strip()} - {h2[1].find("a").text.strip()} {h2[2].find("a").text.strip()}'
    
    # Get decryption key
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    part2 = df['–––––– Part 2 ––––––']['Rank']
    
    decryption_key = list(part2.replace("–", -1).astype(int))
    return info, decryption_key

info, decryption_key = scrape_key(549)
info

'JamesWordie - Batch #523 London'

# Part 1

In [5]:
_, key = scrape_key(1)
prod_dec_key = reduce(lambda x,y: x*y, key)
prod_dec_key

9913814436643373168640000000

# Part 2

In [6]:
def get_cipher(sender_identifier, decryption_key):
    """Product of all decryption key elements,
    add or subtract sender_identifier,
    use every 3rd character"""
    
    cipher = reduce(lambda x,y: x*y, decryption_key)
    
    if sender_identifier % 2 == 0:
        cipher += sender_identifier
    else:
        cipher -= sender_identifier
    
    cipher = int(str(abs(cipher))[0::3])
    
    return cipher

get_cipher(549, decryption_key)

3736562624

In [7]:
def process_ternary(ternary: str, message):
    """Uses the ternary number to organize the message"""
    
    message = list(message)
    for i,c in enumerate(ternary[::-1]):
        if c == '1':
            # shift left
            message.append(message.pop(0))
        elif c == '2':
            # shift right
            message = list(np.roll(message, int('1'+'0'*i,3)))
        else:
            # reverse
            message = message[::-1]
    
    return ''.join(message)

process_ternary('1201', "abcde")

'dcbae'

In [8]:
def get_ternary(cipher):
    """Converts integer to ternary"""
    
    return np.base_repr(cipher,base=3)

get_ternary(46)

'1201'

### Generate the rudolphabet from the supplied lyrics

In [9]:
# Rudolphabet
lyrics = """You know Dasher, and Dancer, and
Prancer, and Vixen,
Comet, and Cupid, and
Donder and Blitzen
But do you recall
The most famous reindeer of all

Rudolph, the red-nosed reindeer
had a very shiny nose
and if you ever saw it
you would even say it glows.

All of the other reindeer
used to laugh and call him names
They never let poor Rudolph
play in any reindeer games.

Then one foggy Christmas eve
Santa came to say:
"Rudolph with your nose so bright,
won't you guide my sleigh tonight?"

Then all the reindeer loved him
as they shouted out with glee,
Rudolph the red-nosed reindeer,
you'll go down in history!"""
standard = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
extras = "Çàéè0123456789 .!?,;:'-#/\()"

In [10]:
# Generate rudolphabet
rudolphabet = ""
for c in lyrics:
    if c not in rudolphabet and c in standard:
        rudolphabet += c

for c in standard:
    if c not in rudolphabet:
        rudolphabet += c

rudolphabet += extras
rudolphabet

"YouknwDasherdcPVixCmtpBlzyTfRvgASbEFGHIJKLMNOQUWXZjqÇàéè0123456789 .!?,;:'-#/\\()"

In [11]:
def get_prime_factors(cipher):
    """Returns a list of prime factors from an integer"""
    i = 2
    factors = []
    while i * i <= cipher:
        if cipher % i:
            i += 1
        else:
            cipher //= i
            factors.append(i)
    if cipher > 1:
        factors.append(cipher)
        
    return factors

get_prime_factors(46)

[2, 23]

In [12]:
len(rudolphabet)

80

In [13]:
def decipher(factors, message):
    """Uses Caesar cipher and the rudolphabet to decipher message"""
    shift = sum(factors)
    if len(factors) == 1:
        shift += 5
        
    length = len(rudolphabet)
    while shift >= length:
        shift -= length
        
    shifted_rudolphabet = rudolphabet[shift:] + rudolphabet[:shift]

    return message.translate(str.maketrans(rudolphabet, shifted_rudolphabet))

decipher([2, 23], "abcde")

'S2IHF'

In [18]:
def part_2(inp):
    """Loops over each input and executes required functions.
    Returns dictionary with sender info and decrypted message"""
    
    results = {}
    for sender_identifier, message in inp.items():
        
        info, decryption_key = scrape_key(sender_identifier)
        print(f"{decryption_key=}")
        
        cipher = get_cipher(sender_identifier, decryption_key)
        print(f"{cipher=}")
        
        ternary = get_ternary(cipher)
        print(f"{ternary=}")
        
        new_message = process_ternary(ternary, message)
        print(f"{new_message=}")
        
        factors = get_prime_factors(cipher)
        print(f"{factors=}")
        
        result = decipher(factors, new_message)
        print(f"{result=}")
        
        results[info] = result
        
    return results

In [15]:
out = part_2(inp)

### Write results to file

In [16]:
with open("out.txt", "w") as file:
    for k,v in out.items():
        file.write(f"{k}\n>>> {v}\n---\n")


In [19]:
part_2({323: inp[323]})

decryption_key=[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 68, 110, 95, 90, 110, 101, 80, 84, 80, 98, 72]
cipher=25374397
ternary='1202202011010111'
new_message="#(?u:-jm:aX/(w78ÇXu#uX.(u;Xu;:oX? -X,u?aX( Xu78#;ZXs;:X.7'9Xa78Xk8uX( u7X7'd? (D( dXu;(,Xh7'X8,X(,Xd':?uwaX?kk':"
factors=[1867, 13591]
result='ciated!Hey Pilou, tct with them and stay in touch. The work you put into organizing this for us is greatly appre'


{'ThomasGerleve - Batch #736 Cologne': 'ciated!Hey Pilou, tct with them and stay in touch. The work you put into organizing this for us is greatly appre'}